In [55]:
import math
import random
import os
from pprint import pprint

#### Configuration (Modify here only)

In [56]:
# filename = "D:/VLSI/Capstone/tb/v2/top_tb.sv"
# root_path = r"D:/VLSI/Capstone"

# filename = r"\\wsl$\Ubuntu-22.04\home\hwanzar\capstone241\graph-attention-network-fpga\tb\top_tb.sv"
# root_path = r"\\wsl$\Ubuntu-22.04\home\hwanzar\capstone241\graph-attention-network-fpga"

filename = r"/home/hwanzar/capstone241/graph-attention-network-fpga/tb/v2/top_tb.sv"
root_path = r"/home/hwanzar/capstone241/graph-attention-network-fpga"

In [57]:
params = {
  "H_NUM_OF_ROWS" : 100,
  "H_NUM_OF_COLS" : 10,         # WARNING: Do not set as powers of 2.

  "W_NUM_OF_ROWS" : 10,
  "W_NUM_OF_COLS" : 16,

  "NUM_OF_NODES"  : 6,          # WARNING: Do not set as powers of 2.
}

#### 1. Random Inputs

In [58]:
params.update({
  "COL_IDX_DEPTH"   : 0,                                # số lượng col_idx tổng cộng
  "VALUE_DEPTH"     : 0,                                # số lượng value tổng cộng
  "NODE_INFO_DEPTH" : 0,                                # số lượng node tổng cộng
  "WEIGHT_DEPTH"    : 0,                                # = W_ROWS * W_COLS
  "WH_DEPTH"        : 0,                                # = số lượng node tổng cộng
  "A_DEPTH"         : params["W_NUM_OF_COLS"] * 2,      # = W_COLS * 2
})

In [59]:
def random_matrix(rows, cols, sparsity_percent):
	total_elements = rows * cols
	num_zeros = int(total_elements * sparsity_percent / 100)
	num_non_zeros = total_elements - num_zeros
	matrix_elements = [0] * num_zeros + [random.randint(1, 5) for _ in range(num_non_zeros)]
	random.shuffle(matrix_elements)
	sparse_matrix = []
	for i in range(rows):
		sparse_matrix.append(matrix_elements[i * cols:(i + 1) * cols])
	for row in sparse_matrix:
		if all(value == 0 for value in row):
			row[random.randint(0, cols - 1)] = random.randint(1, 9)
	return sparse_matrix

##### Weight


In [60]:
weight = random_matrix(params["W_NUM_OF_ROWS"], params["W_NUM_OF_COLS"], 0)
# weight = [[1] * params["W_NUM_OF_COLS"] for _ in range(params["W_NUM_OF_ROWS"])]

##### Weight Vector a

In [61]:
a = [1 if i < params["A_DEPTH"] / 2 else 2 for i in range(params["A_DEPTH"])]
# a = random_matrix(1, params["A_DEPTH"], 0)[0]

##### SubGraph

In [62]:
subGraph = []
sub_row = random.randint(2, params["NUM_OF_NODES"])
sub_cnt = 0
source_node_list = []
while sub_cnt < params["H_NUM_OF_ROWS"]:
  sub_row = random.randint(2, params["NUM_OF_NODES"])
  sub_cnt += sub_row
  if sub_cnt >= params["H_NUM_OF_ROWS"]:
    sub_row = sub_row - (sub_cnt - params["H_NUM_OF_ROWS"])
    if sub_row == 1:
      sub_row = 2
      for item in subGraph:
        if len(item) >= 3:
          item.pop(-1)
          break
  subG = random_matrix(sub_row, params["H_NUM_OF_COLS"], 50)

  subGraph.append(subG)

print("Num of subGraph = ", len(subGraph))

Num of subGraph =  22


##### Feature

In [63]:
feature = []
for i in range(len(subGraph)):
  feature += subGraph[i]

#### 2. Calculation

##### SPMM

In [64]:
wh = [[0 for _ in range(len(weight[0]))] for _ in range(len(feature))]
for i in range(len(feature)):
  for j in range(len(weight[0])):
    for k in range(len(weight)):
      wh[i][j] += feature[i][k] * weight[k][j]

def matrix_multiplication(subgraph, weight_matrix):
  rows = len(subgraph)
  cols = len(weight_matrix[0])
  common_dim = len(weight_matrix)

  result_matrix = [[0 for _ in range(cols)] for _ in range(rows)]

  for i in range(rows):
    for j in range(cols):
      for k in range(common_dim):
        result_matrix[i][j] += subgraph[i][k] * weight_matrix[k][j]
  return result_matrix

WH = []
for i in range(len(subGraph)):
	wh_temp = matrix_multiplication(subGraph[i], weight)
	WH.append(wh_temp)

##### DMVM

In [65]:
midpoint = len(a) // 2
a_1 = a[:midpoint]
a_2 = a[midpoint:]

dmvm = []
coef = []

for wh in WH:
  subSum = 0
  for idx, row in enumerate(wh):
    sum = 0
    for i in range(len(row)):
      if (idx == 0):
        sum += row[i] * a_1[i]
      else:
        sum += row[i] * a_2[i]
    dmvm.append(sum)
    if (idx == 0):
      subSum = sum
    coef.append(sum+subSum)

##### Softmax

In [66]:
# TODO:

##### Aggregator

In [67]:
# TODO:

#### 3. GCSR structure

In [68]:
source_node_list = []
num_nodes_subgraph = []
node_info = []
col_idx = []
value = []
non_zero_ele = []

row_length_bits = math.ceil(math.log2(params["H_NUM_OF_COLS"]))
num_of_nodes_bits = math.ceil(math.log2(params["NUM_OF_NODES"]))

for i in range(len(subGraph)):
  source_node = subGraph[i][0]
  num_node = len(subGraph[i])
  source_node_list.append(source_node)
  num_nodes_subgraph.append(num_node)
  node_info_list = []
  for j in range(len(subGraph[i])):
    if j == 0:
      flag = '1'
    else:
      flag = '0'
    non_zero_values = [ele for k, ele in enumerate(subGraph[i][j]) if ele != 0]
    # print("Non Zero Values = ", non_zero_values)
    row_length = len(non_zero_values)
    node_info_bin = (f"{row_length:0{row_length_bits}b}"f"{num_node:0{num_of_nodes_bits}b}" f"{flag}")
    node_info_list.append(node_info_bin)

  node_info.append(node_info_list)

def decode_node_info(node_info_string):
  # Extract each part from the string
  flag_bits = 1
  row_length = int(node_info_string[:row_length_bits], 2)
  num_of_nodes = int(node_info_string[row_length_bits:row_length_bits+num_of_nodes_bits], 2)

  flag = node_info_string[-flag_bits:]  # '1' for source node, '0' for neighbor node

  # Return a dictionary for better readability
  return {
    'row_length': row_length,
    'num_of_nodes': num_of_nodes,
    'is_source_node': flag == '1'
  }

# Example usage
decoded_node_info = []
for nd_info in node_info:
  decoded_info = [decode_node_info(info) for info in nd_info]
  decoded_node_info.append(decoded_info)
# pprint(decoded_node_info)

In [69]:
def extract_from_raw(raw_list):
	col_idx = []
	value = []
	node_info = []
	for row in raw_list:
		non_zero_elements = [(j, ele) for j, ele in enumerate(row) if ele != 0]
		col_idx.extend([idx for idx, _ in non_zero_elements])
		value.extend([val for _, val in non_zero_elements])
		row_len = len(non_zero_elements)

	return col_idx, value

#### 4. SubGraph Dictionary

In [70]:
GRAPH = {}
DMVM = []
COEF_I = []
idx = 0
for subgraph in subGraph:
  dmvm_temp = []
  coef_temp = []
  for i in range(len(subgraph)):
    dmvm_temp.append(dmvm[idx])
    coef_temp.append(coef[idx])
    idx += 1
  DMVM.append(dmvm_temp)
  COEF_I.append(coef_temp)
for i in range(len(subGraph)):
  col_idx_, value_ = extract_from_raw(subGraph[i])
  GRAPH[i] = {"raw" : subGraph[i], "col_idx" : col_idx_, "value" : value_, "node_info" : node_info[i], "WH": WH[i],  "DMVM": DMVM[i], "Coef": COEF_I[i]}

colors = {
  "raw": "\033[91m",       # Red
  "col_idx": "\033[92m",   # Green
  "value": "\033[93m",     # Yellow
  "node_info": "\033[94m", # Blue
  "WH": "\033[95m",        # Magenta
  "DMVM": "\033[96m",      # Cyan
  "Coef": "\033[97m",      # White
  "reset": "\033[0m"       # Reset color
}

for subgraph in GRAPH.values():
  print("-" * 90)
  for i in range(len(subgraph["raw"])):
    if i == 0:
      print(f"{colors['raw']}raw\t\t{subgraph['raw'][i]}{colors['reset']}")
    else:
      print(f"{colors['raw']}\t\t{subgraph['raw'][i]}{colors['reset']}")

  print(f"{colors['col_idx']}col_idx\t\t{subgraph['col_idx']}{colors['reset']}")
  print(f"{colors['value']}value\t\t{subgraph['value']}{colors['reset']}")
  print(f"{colors['node_info']}node_info\t{subgraph['node_info']}{colors['reset']}")

  for i in range(len(subgraph["WH"])):
    if i == 0:
      print(f"{colors['WH']}SPMM\t\t{subgraph['WH'][i]}{colors['reset']}")
    else:
      print(f"{colors['WH']}\t\t{subgraph['WH'][i]}{colors['reset']}")

  print(f"{colors['DMVM']}DMVM\t\t{subgraph['DMVM']}{colors['reset']}")
  print(f"{colors['Coef']}Coef\t\t{subgraph['Coef']}{colors['reset']}")
  print("-" * 90)

------------------------------------------------------------------------------------------
raw		[1, 0, 0, 4, 0, 0, 0, 1, 1, 3]
		[1, 2, 0, 4, 4, 1, 0, 2, 0, 0]
		[0, 2, 5, 0, 3, 3, 0, 4, 2, 3]
		[0, 0, 0, 0, 0, 0, 0, 0, 2, 2]
col_idx		[0, 3, 7, 8, 9, 0, 1, 3, 4, 5, 7, 1, 2, 4, 5, 7, 8, 9, 8, 9]
value		[1, 4, 1, 1, 3, 1, 2, 4, 4, 1, 2, 2, 5, 3, 3, 4, 2, 3, 2, 2]
node_info	['01011001', '01101000', '01111000', '00101000']
SPMM		[28, 22, 15, 41, 34, 21, 23, 28, 43, 29, 39, 32, 22, 24, 41, 31]
		[26, 25, 26, 52, 61, 31, 31, 42, 49, 30, 36, 42, 24, 49, 42, 51]
		[50, 62, 56, 86, 67, 62, 68, 71, 73, 71, 58, 71, 65, 67, 71, 71]
		[16, 14, 10, 12, 10, 10, 16, 10, 14, 10, 14, 8, 14, 8, 18, 12]
DMVM		[473, 1234, 2138, 392]
Coef		[946, 1707, 2611, 865]
------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------
raw		[0, 1, 0, 2, 0, 5, 0, 0, 4, 2]
		[0, 0, 4, 5, 0, 4, 3, 0, 0, 5

#### 5. Update testbench

##### Inputs

In [71]:
file_path_input = os.path.join(root_path, "tb/inputs")

In [72]:
wrt_col_idx = []
wrt_value = []
wrt_node_info = []

for key, value in GRAPH.items():
  wrt_col_idx.append(value["col_idx"])
  wrt_value.append(value["value"])
  wrt_node_info.append(value["node_info"])

In [73]:
def write_list_to_file(file_path, data_list):
  with open(file_path, 'w') as file:
    print(f" open {file_path}" )
    for item in data_list:
      for it in item:
        file.write(f"{it}\n")
def write_list_to_file2(file_path, data_list):
  with open(file_path, 'w') as file:
    print(f" open {file_path}" )
    for item in data_list:
      file.write(f"{item}\n")

write_list_to_file(file_path_input + "/col_idx.txt", wrt_col_idx)
write_list_to_file(file_path_input + "/value.txt", wrt_value)
write_list_to_file(file_path_input + "/node_info.txt", wrt_node_info)
write_list_to_file(file_path_input + "/weight.txt", weight)
write_list_to_file2(file_path_input + "/a.txt", a)

 open /home/hwanzar/capstone241/graph-attention-network-fpga/tb/inputs/col_idx.txt
 open /home/hwanzar/capstone241/graph-attention-network-fpga/tb/inputs/value.txt
 open /home/hwanzar/capstone241/graph-attention-network-fpga/tb/inputs/node_info.txt
 open /home/hwanzar/capstone241/graph-attention-network-fpga/tb/inputs/weight.txt
 open /home/hwanzar/capstone241/graph-attention-network-fpga/tb/inputs/a.txt


In [ ]:
#14/11/2024
print(WH)
file_path_output = os.path.join(root_path, "tb/outputs/")
print(file_path_output)
wh_output = []
for _WH in WH:
  for _wh in _WH:
    for wh in _wh:
      wh_output.append(wh)
print(wh_output)

write_list_to_file2(file_path_output + "/WH.txt", wh_output)

[[[28, 22, 15, 41, 34, 21, 23, 28, 43, 29, 39, 32, 22, 24, 41, 31], [26, 25, 26, 52, 61, 31, 31, 42, 49, 30, 36, 42, 24, 49, 42, 51], [50, 62, 56, 86, 67, 62, 68, 71, 73, 71, 58, 71, 65, 67, 71, 71], [16, 14, 10, 12, 10, 10, 16, 10, 14, 10, 14, 8, 14, 8, 18, 12]], [[38, 42, 49, 55, 48, 29, 34, 47, 55, 44, 42, 37, 47, 40, 54, 47], [53, 47, 57, 97, 52, 53, 46, 76, 77, 84, 76, 69, 62, 49, 89, 70], [36, 19, 23, 39, 33, 24, 21, 33, 47, 22, 23, 25, 29, 23, 42, 38], [16, 14, 12, 30, 32, 16, 12, 21, 33, 17, 24, 27, 14, 23, 21, 21], [60, 56, 71, 77, 60, 47, 48, 63, 78, 58, 57, 53, 73, 64, 64, 71], [29, 30, 29, 44, 28, 19, 24, 27, 46, 36, 37, 31, 36, 37, 32, 40]], [[52, 49, 64, 82, 60, 47, 54, 73, 74, 68, 78, 59, 74, 71, 67, 74], [26, 17, 43, 40, 20, 23, 17, 42, 30, 35, 35, 23, 41, 28, 34, 35], [56, 57, 42, 83, 62, 60, 66, 57, 76, 63, 62, 70, 58, 68, 65, 73], [30, 21, 27, 31, 23, 28, 17, 19, 30, 16, 8, 23, 23, 23, 22, 30]], [[23, 38, 36, 68, 60, 27, 34, 52, 63, 56, 59, 55, 41, 58, 48, 53], [18, 

##### Parameters

In [75]:
_col_idx = [len(item) for item in wrt_col_idx]
len_col_idx = 0
for col in _col_idx:
  len_col_idx += col

_value = [len(item) for item in wrt_value]
len_value = 0
for val in _value:
  len_value += val

_node_info = [len(item) for item in wrt_node_info]
len_node_info = 0
for node in _node_info:
  len_node_info += node

params["COL_IDX_DEPTH"] = len_col_idx
params["VALUE_DEPTH"] = len_value
params["NODE_INFO_DEPTH"] = len_node_info
params["WEIGHT_DEPTH"] = params["W_NUM_OF_ROWS"] * params["W_NUM_OF_COLS"]
params["WH_DEPTH"] = len_node_info
params["A_DEPTH"] = params["W_NUM_OF_COLS"] * 2
pprint(params)

{'A_DEPTH': 32,
 'COL_IDX_DEPTH': 500,
 'H_NUM_OF_COLS': 10,
 'H_NUM_OF_ROWS': 100,
 'NODE_INFO_DEPTH': 100,
 'NUM_OF_NODES': 6,
 'VALUE_DEPTH': 500,
 'WEIGHT_DEPTH': 160,
 'WH_DEPTH': 100,
 'W_NUM_OF_COLS': 16,
 'W_NUM_OF_ROWS': 10}


In [76]:
def update_parameters(filename, parameters):
  with open(filename, 'r') as file:
    file_data = file.readlines()

  for i, line in enumerate(file_data):
    for param, new_value in parameters.items():
      found_param = "parameter " + param
      # print(param)
      if found_param in line:
        file_data[i] = f"  parameter {param}\t\t\t= {new_value},\n"

  with open(filename, 'w') as file:
      file.writelines(file_data)

update_parameters(filename, params)

##### Root Path

In [77]:
def update_root_path(filename, root_path):
  with open(filename, 'r') as file:
    file_data = file.readlines()

  for i, line in enumerate(file_data):
    if "localparam string ROOT_PATH" in line:
      file_data[i] = f"\tlocalparam string ROOT_PATH = \"{root_path}\";\n"

  with open(filename, 'w') as file:
      file.writelines(file_data)

update_root_path(filename, root_path)